In [1]:
import pandas as pd 
import numpy as np 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns',None)

In [2]:
df_home=pd.read_csv('/Users/aurelientarroux/Desktop/Data_Challenge_QRT/Data/Train_Data/train_home_team_statistics_df.csv')
df_away=pd.read_csv('/Users/aurelientarroux/Desktop/Data_Challenge_QRT/Data/Train_Data/train_away_team_statistics_df.csv')

df_home.columns='home_'+df_home.columns
df_away.columns='away_'+df_away.columns

df_result=pd.read_csv('/Users/aurelientarroux/Desktop/Data_Challenge_QRT/Data/Y_train_1rknArQ.csv')



In [3]:
base_match=pd.merge(df_home,df_away,left_on="home_ID",right_on='away_ID',how='inner')

base=pd.merge(base_match,df_result,left_on="home_ID",right_on='ID',how='inner')

base=base.drop(['home_ID','home_LEAGUE','home_TEAM_NAME','away_ID','away_LEAGUE','away_TEAM_NAME'],axis=1)

base.dropna(inplace=True)

In [4]:
def resultat(row):
    if row['HOME_WINS']==1:
        return 'HOME'
    elif row['AWAY_WINS']==1:
        return 'AWAY'
    else:
        return 'DRAW'

base['Resultat']=base.apply(resultat, axis=1)

base=base.drop(['HOME_WINS','AWAY_WINS','DRAW'],axis=1)


In [5]:
X = base.drop(['ID'],axis=1)

train, test = train_test_split(X, test_size=0.2, random_state=42)

scaler = StandardScaler()
scaler.fit(train.iloc[:,0:279])
train.iloc[:,0:279] = scaler.transform(train.iloc[:,0:279])
test.iloc[:,0:279]= scaler.transform(test.iloc[:,0:279])

In [6]:
def distance(num_ligne_train,num_ligne_test):
    distance=0
    for k in range(179):
        distance+= ( train.iloc[num_ligne_train,k] - test.iloc[num_ligne_test,k] )**2
    return np.sqrt(distance)


In [7]:
K_NN=int(np.sqrt(len(train)))

compte_juste=0
compte=0

for k in range(len(test)):
    
    list_distance=[]
    for i in range(len(train)):
        list_distance+=[[distance(i,k),train.iloc[i,280]]]
    list_distance.sort(key=lambda x:x[0])
            
    list_class=[]
    for j in range(K_NN):
        list_class+=[list_distance[j][1]]
    element_plus_frequent = max(list_class, key=list_class.count)
    
    compte+=1
    if test.iloc[k,280]==element_plus_frequent:
        compte_juste+=1
    
    print(test.iloc[k,280],element_plus_frequent )
    print(compte_juste/compte)



KeyboardInterrupt: 